# Solving a 4x4 grid cell Sudoku with the latest D-Wave Quantum Annealer via Cloud (D-Wave leap)

In [1]:
# for classical solver (simulated annealing)
import dimod
import operator
import dwavebinarycsp
import numpy as np
import math

In [2]:
from quantum_annealing_sudoku import quantum_annealing_sudoku
from quantum_annealing_sudoku.label_encoder import encode_var_labels, decode_var_labels

In [3]:
sudoku_4x4 = quantum_annealing_sudoku.QuantumAnnealingSudoku(grid_9x9 = False)


#get some function shortcuts for better readability 
check_sudoku = sudoku_4x4.check_sudoku
encode_board_to_binary = sudoku_4x4.encode_board_to_binary
decode_board_from_binary = sudoku_4x4.decode_board_from_binary
print_board = sudoku_4x4.print_board

In [4]:
board = ((1,2,0,0),
         (0,0,0,0),
         (0,0,1,0),
         (0,0,4,0),
)
sudoku_4x4.print_board((board))

 1  2 | -  - 
 -  - | -  - 
------|------
 -  - | 1  - 
 -  - | 4  - 



In [5]:
correct_board = ((1,2,3,4),
         (4,3,2,1),
         (2,1,4,3),
         (3,4,1,2),
)
sudoku_4x4.print_board(correct_board)

 1  2 | 3  4 
 4  3 | 2  1 
------|------
 2  1 | 4  3 
 3  4 | 1  2 



In [6]:
# check if the solution check works
print('Error Count: ',sudoku_4x4.check_sudoku(correct_board))
print('Error Count: ',sudoku_4x4.check_sudoku(board))

Error Count:  0
Error Count:  6


In [7]:
constant = 0
N = 4
cell_qubo = {}
linear = {}
quad = {}
penalty_weight = 1
for i in range(1,N+1):
        for j in range(1,N+1):
            for k1 in range(1,N+1):  
                    var_1 = encode_var_labels(i,j,k1)
                    for k2 in range(1,N+1):
                            var_2 = encode_var_labels(i,j,k2)
                            if var_1 == var_2:
                                linear[var_1] = -1*penalty_weight
                            else:
                                quad[var_1,var_2]= 2*penalty_weight
                        #linear[var_1] = 2
                    
            constant+=1
            

cell_qubo[()] = constant*penalty_weight
cell_qubo["linear"] = linear
cell_qubo["quadratic"] = quad

#bqm = dimod.BinaryQuadraticModel(cell_qubo,constant,dimod.Vartype.BINARY)

In [8]:
penalty_weight=1
constant = 0
N = 4
column_qubo = {}
lin_column={}
quad_column={}
binary_board = sudoku_4x4.encode_board_to_binary(board)
for k in range(1,N):
        for j in range(1,N):
            for i1 in range(1,N):
                var_1 = encode_var_labels(i1,j,k)
                for i2 in range(1,N):               
                    var_2 = encode_var_labels(i2,j,k)
                    if var_1 == var_2:
                        lin_column[var_1] = -1*penalty_weight
                    else:
                        quad_column[var_1,var_2] = 2*penalty_weight
        constant+=1
        
column_qubo[()] = constant*penalty_weight
column_qubo['linear'] = lin_column
column_qubo['quadratic'] = quad_column

In [9]:
constant = 0
N = 4
row_qubo = {}
linear_row ={}
quadratic_row={}
penalty_weight=2
binary_board = sudoku_4x4.encode_board_to_binary(board)
for k in range(1,N):
        for i in range(1,N):
            for j1 in range(1,N):
                var_1 = encode_var_labels(i,j1,k)
                for j2 in range(1,N):      
                    var_2 = encode_var_labels(i,j2,k)
                    if var_1 == var_2:
                        linear_row[var_1] = -1*penalty_weight
                    else:
                        quadratic_row[var_1, var_2] = 2*penalty_weight
        constant+=1
        
row_qubo[()] = constant*penalty_weight
row_qubo['linear'] = linear_row
row_qubo['quadratic'] = quadratic_row

In [10]:
binary_board = sudoku_4x4.encode_board_to_binary(board)
hint = []
penalty_weight = 4

for k, color in enumerate(binary_board):
    for i, row in enumerate(color):
        for j, cell in enumerate(row):
            if cell>0:
                hint.append([i+1,j+1,k+1])
                
hint_qubo_linear = {}
hint_qubo = {}
constant=0
for (i, j, k) in hint:
    re_label = encode_var_labels(i,j,k) 
    hint_qubo_linear[re_label] = -1*penalty_weight
    constant += 1

hint_qubo[()] = constant*penalty_weight
hint_qubo['linear'] = hint_qubo_linear

In [11]:
from math import sqrt
constant = 0
N = 4
duplicate_qubo = {}
dupli_linear = {}
dupli_quadr = {}
sqrtN = int(sqrt(N))
for grid_i in range(sqrtN):
    for grid_j in range(sqrtN):
        for k in range(N):
            # there can be only one k in the same subgrid.
            for i1 in range(grid_i * 2, grid_i * 2 + 2):
                for j1 in range(grid_j * 2, grid_j * 2 + 2):
                    for i2 in range(grid_i * 2, grid_i * 2 + 2):
                        var_1 = encode_var_labels(i1+1,j1+1,k+1)
                        for j2 in range(grid_j * 2, grid_j * 2 + 2):
                            var_2 = encode_var_labels(i2+1,j2+1,k+1)
                            if var_1 == var_2:
                                dupli_linear[var_1] = -1
                            else:
                                dupli_quadr[var_1,var_2] = 1
            constant+=1

            
duplicate_qubo[()] = constant
duplicate_qubo['linear'] = dupli_linear
duplicate_qubo['quadratic'] = dupli_quadr

In [12]:
def append_linear(result_qubo_linear, qubo):    
    if not qubo.get('linear'):
        return result_qubo_linear
    for index, value in qubo['linear'].items():
        if result_qubo_linear.get(index):
            result_qubo_linear[index] += value     
        else:
            result_qubo_linear[index] = value
            
    return result_qubo_linear
            
def append_quadratic(result_qubo_quadratic, qubo):    
    if not qubo.get('quadratic'):
        return result_qubo_quadratic
    for index, value in qubo['quadratic'].items():
        if result_qubo_quadratic.get(index):
            result_qubo_quadratic[index] += value 
        else:
            result_qubo_quadratic[index] = value
            
    return result_qubo_quadratic
            

In [13]:
# qubos that are needed to achieve the constraints
qubos=[cell_qubo, column_qubo, row_qubo, hint_qubo, duplicate_qubo]

In [14]:
result_qubo_linear = {}
result_qubo_quadratic = {}
for qubo in qubos:    
    if qubo is not None:
        result_qubo_linear = append_linear(result_qubo_linear, qubo)
        result_qubo_quadratic = append_quadratic(result_qubo_quadratic, qubo)

In [15]:
constant= cell_qubo.get((),0) + hint_qubo.get((),0) + row_qubo.get((),0) + column_qubo.get((),0) + duplicate_qubo.get((),0)
bqm = dimod.BinaryQuadraticModel(result_qubo_linear,result_qubo_quadratic, constant,dimod.Vartype.BINARY)

In [16]:
# test run
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_set=sampler.sample(bqm, num_reads=30,num_sweeps=2222)


In [152]:
from random import randrange
import time

def find_optimal_solution(best_solution_global, current_solution, current_energy, 
                          sa_sample_set):
    best_solution = {}
    error_count=20
    iteration = 0
    sampler = neal.SimulatedAnnealingSampler()
    start_time = time.perf_counter()
    while error_count>0:
        """
        sample_set = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2],
                                        num_reads=10, num_sweeps=20000,
                                       beta_schedule_type='geometric')
        """
        random_seed = randrange(9000)
        #random_seed = 5832
        num_reads = randrange(12000)
        num_sweeps = randrange(60000)
        num_reads = 5
        num_sweeps=50
        #num_sweeps*=iteration
        pre_anneal = time.perf_counter()
        sample_set=sampler.sample(bqm,seed=random_seed ,num_reads=num_reads,num_sweeps=num_sweeps)
        post_anneal = time.perf_counter()
        annealing_time = post_anneal-pre_anneal
        iteration+=1
        sa_sample = sample_set.copy()
        sa_sample.info.update({'annealing_time':annealing_time})
        sa_sample_set.append(sa_sample)


        for solution, energy in sample_set.data(['sample', 'energy']):
            binary_solution_board= np.zeros((4, 4, 4))
            for index, value in solution.items():
                if type(index) is int and index>0:
                    board_index = decode_var_labels(index)
                    binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
            solution_board=decode_board_from_binary(binary_solution_board)
            error_count_temp = check_sudoku(solution_board)
            
            
            
            current_solution.append(1)
            current_energy.append(energy)
            current_solution[:] = []
            current_energy[:] = []
            current_solution.append(solution)
            current_energy.append(energy)
            
            
        overall_time = time.perf_counter()-start_time
        if error_count_temp<error_count:
            best_solution = solution_board
            best_solution_global.append(solution)
            error_count=error_count_temp
            print("\nError Count:",error_count, "iteration:", iteration, "Energy:", energy,
                  "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps, 
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            
        if iteration%100 == 0:
            print("\nCurrent State:",
                  "Error Count:",error_count_temp, "iteration:", iteration, "Energy:", energy,
                 "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps,
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            

            
    return sample_set
        

In [153]:
import multiprocessing
manager = multiprocessing.Manager()
best_solution = manager.list()
current_solution = manager.list()
current_energy = manager.list()
sa_sample_set = manager.list()

process = multiprocessing.Process(target=find_optimal_solution, 
            
                                  args= (best_solution, current_solution, 
                                         current_energy, sa_sample_set))
process.start()




Error Count: 3 iteration: 1 Energy: -8.0 Seed: 4130 num_reads: 5 sweeps: 50 Annealing Time: 0.009436460008146241 Overall Time: 0.020352824998553842

Error Count: 2 iteration: 2 Energy: -12.0 Seed: 3694 num_reads: 5 sweeps: 50 Annealing Time: 0.004648133006412536 Overall Time: 0.08536536802421324

Error Count: 1 iteration: 5 Energy: -4.0 Seed: 1924 num_reads: 5 sweeps: 50 Annealing Time: 0.004066224995767698 Overall Time: 0.18412541100406088

Error Count: 0 iteration: 7 Energy: -8.0 Seed: 5755 num_reads: 5 sweeps: 50 Annealing Time: 0.004542035021586344 Overall Time: 0.255865854996955


In [154]:
import time
time.sleep(2)
#wait with process terminating; in order to have  
#results when doing a (quick) autorun

process.terminate()
process

<Process name='Process-20' pid=1766974 parent=1750564 stopped exitcode=0>

In [155]:
#sample_set = list(current_solution)
sample_set_simulated_annealing = list(best_solution)

In [176]:
sa_sample_set[0].info

{'beta_range': [0.037467415165402446, 9.210340371976184],
 'beta_schedule_type': 'geometric',
 'annealing_time': 0.009436460008146241}

In [175]:
solution={}
solution_2={}
solution_3={}
for number, solution in enumerate(sample_set_simulated_annealing):
    if number==0:
        solution = solution
    elif number==1:
        solution_2 = solution
    elif number==2:
        solution_3 = solution
        
    
last_solution = solution

In [159]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value


In [160]:
solution_board=decode_board_from_binary(binary_solution_board)
solution_board

array([[1., 3., 2., 4.],
       [4., 2., 3., 1.],
       [2., 4., 1., 3.],
       [3., 1., 4., 2.]])

In [161]:
print_board(board)

 1  2 | -  - 
 -  - | -  - 
------|------
 -  - | 1  - 
 -  - | 4  - 



In [162]:
sudoku_4x4.print_board(solution_board)
print("Error Count:",sudoku_4x4.check_sudoku(solution_board))
print("BQM Energy:", bqm.energy(solution))

 1  3 | 2  4 
 4  2 | 3  1 
------|------
 2  4 | 1  3 
 3  1 | 4  2 

Error Count: 0
BQM Energy: -8.0


In [163]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in solution_2.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
sudoku_4x4.print_board(solution_board)
print("Error Count:",sudoku_4x4.check_sudoku(solution_board))

 1  3 | 2  4 
 4  2 | 3  1 
------|------
 2  4 | 1  3 
 3  1 | 4  2 

Error Count: 0


In [164]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_3.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 1  3 | 2  4 
 4  2 | 3  1 
------|------
 2  4 | 1  3 
 3  1 | 4  2 

Error Count: 0


In [165]:
bqm.energy(solution)

-8.0

In [188]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in sa_sample_set[0].first.sample.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 1  3 | 2  4 
 4  2 | 3  1 
------|------
 2  4 | 1  3 
 3  1 | 4  2 

Error Count: 0


In [199]:
sa_sample_df = sa_sample_set[0].to_pandas_dataframe()
annealing_time = sa_sample_set[0].info['annealing_time']
sa_sample_df['annealing_time'] = annealing_time

In [200]:
sa_sample_df.head(2)

,1,2,3,4,10,11,12,13,19,20,...,263,264,265,271,272,273,274,energy,num_occurrences,annealing_time
0,1,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,-9.0,1,0.009436
1,1,0,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,-8.0,1,0.009436


In [202]:
sa_sample_df.to_csv('./data/results_simulated_annealing/'+
                    'simulated_annealing_'+ str(annealing_time)+
                   '.csv')

In [203]:
bqm_graph = bqm.to_networkx_graph()

In [204]:
print('Number of problem nodes (variables):',len(bqm_graph.nodes))
print('Number of problem edges (couplings):',len(bqm_graph.edges))

Number of problem nodes (variables): 64
Number of problem edges (couplings): 228


Turns out the number of edges and nodes is feasible (small) enough to be embedded on the quantum annealing hardware!

In [205]:
# imports for D-Wave
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
from dwave.embedding import embed_bqm, unembed_sampleset
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.embedding.chain_breaks import majority_vote

import dwave.inspector


#### Either use the D-Wave embedding composite to ansemble the steps of embedding and sampling or split them up by using minorminor.find_embedding

In [206]:
use_embedding_composite=True

In [207]:
# Use a D-Wave system as the sampler
sampler = DWaveSampler() 
print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system1.1 was selected.


In [208]:
if use_embedding_composite is True: 
    # Set up a D-Wave system EmbeddingComposite as the sampler
    sampler = EmbeddingComposite(sampler)

else:
    # split sampling into embedding first and then sampling
    __, target_edgelist, target_adjacency = sampler.structure
    #bqm = BinaryQuadraticModel.from_qubo(Q)
    emb = find_embedding(bqm.to_qubo()[0], target_edgelist)
    embedded_bqm = embed_bqm(source_bqm=bqm,embedding=emb ,target_adjacency=target_adjacency)
    embedding = {}
    embedding['embedding'] = emb

In [68]:
# sampling on leap hardware
# num_read is very important as it defines the number of sampling 
# made on the hardware

NUM_READS=1
if use_embedding_composite is True:
    result = sampler.sample(bqm, num_reads=NUM_READS)
    d_wave_solution=result.first.sample
else:
    result = sampler.sample(embedded_bqm, num_reads=NUM_READS)
    unembedded_result = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote)
    d_wave_solution = unembedded_result.first.sample
result_df = result.to_pandas_dataframe()
result_df.to_csv('./data/results_quantum_annealing/'
                 +'annealing_run'+result.info['problem_id']+'.csv')

In [69]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in d_wave_solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 1  2 | 3  4 
 4  3 | 2  1 
------|------
 2  4 | 1  3 
 3  1 | 4  2 

Error Count: 0


In [70]:
sample_anneal_time = result.info['timing']['qpu_anneal_time_per_sample']
print('The quantum annealer found a\
valid solution in {} miliseconds !'.format(sample_anneal_time))

The quantum annealer found avalid solution in 20 miliseconds !


### It is possible to solve a 4x4 Sudoku with the Quantum Annealer in constant time!